In [2]:
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix, f1_score,log_loss

# Set the path to your dataset
train_dir = '/data/dataset'
test_dir = '/data/dataset'

# Set the image size and batch size
img_size = (299, 299)
batch_size = 32

# Create a data generator for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the Xception model
base_model = Xception(weights='imagenet', include_top=False)

# Add a new classification layer
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)

predictions = tf.keras.layers.Dense(6, activation='softmax')(x)

# Combine the base model and classification layer
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

labels = ['Foi_Thong', 'Hang_Kra_Rog_Phu_Phan_ST1', 'Hang_Suea_Sakonnakhon_TT1', 'Kroeng_Krawia', 'Tanao_Si_Kan_Khaw_WA1', 'Tanao_Si_Kan_Dang_RD1']

print("\nXception Model")

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model and record validation loss and accuracy after each epoch
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# Save the model
model.save('/code/model/Xception.h5')

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

print("\n")

# Generate predictions for the test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Calculate the confusion matrix and print it
confusion_matrix = confusion_matrix(test_generator.classes, y_pred)
print("Confusion Matrix:\n",confusion_matrix)

num_classes = len(labels)
# Calculate TP, FP, TN, and FN for each class
total_TP, total_FP, total_TN, total_FN = 0, 0, 0, 0
class_losses = []
for i in range(num_classes):
    TP = confusion_matrix[i, i]
    FP = np.sum(confusion_matrix[:, i]) - TP
    FN = np.sum(confusion_matrix[i, :]) - TP
    TN = np.sum(confusion_matrix) - TP - FP - FN
    total_TP += TP
    total_FP += FP
    total_TN += TN
    total_FN += FN
    class_loss = log_loss(test_generator.classes == i, Y_pred[:, i])
    class_losses.append(class_loss)
    print(f"\nClass {i} ({labels[i]}):")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
    print(f"Loss: {class_loss:.3f}")

# Calculate overall accuracy, precision, recall, f1-score, and loss
overall_accuracy = (total_TP + total_TN) / (total_TP + total_FP + total_TN + total_FN)
overall_precision = total_TP / (total_TP + total_FP)
overall_recall = total_TP / (total_TP + total_FN)
overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall)
overall_loss = log_loss(test_generator.classes, Y_pred)

print("\nOverall Performance:")
print(f"Accuracy: {overall_accuracy:.3f}")
print(f"Precision: {overall_precision:.3f}")
print(f"Recall: {overall_recall:.3f}")
print(f"F1-score: {overall_f1:.3f}")
print(f"Loss: {overall_loss:.3f}")

print("\nClass-specific Performance:")
# Calculate precision, recall, f1-score, and loss for each class
for i, label in enumerate(labels):
    class_accuracy = (confusion_matrix[i, i] + np.sum(confusion_matrix) - np.sum(confusion_matrix[i, :]) - np.sum(confusion_matrix[:, i]) + confusion_matrix[i, i]) / np.sum(confusion_matrix)
    class_precision = confusion_matrix[i, i] / np.sum(confusion_matrix[:, i])
    class_recall = confusion_matrix[i, i] / np.sum(confusion_matrix[i, :])
    class_f1 = 2 * class_precision * class_recall / (class_precision + class_recall)
    class_loss = class_losses[i]
    print(f"\nClass {i} ({label}) Performance:")
    print(f"Accuracy: {class_accuracy:.3f}")
    print(f"Precision: {class_precision:.3f}")
    print(f"Recall: {class_recall:.3f}")
    print(f"F1-score: {class_f1:.3f}")
    print(f"Loss: {class_loss:.3f}")
    print("\n")

print("\n")
print("Xeption Model")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
print('Accuracy:', test_acc)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.xticks(range(len(history.history['accuracy'])))
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
print('Loss:', test_loss)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.xticks(range(len(history.history['loss'])))
plt.show()

# Get the weights and biases of the model
weights= model.get_weights()

# Print the shape of the weights and biases
print("Weights shape:", np.shape(weights))
# print("Biases shape:", np.shape(biases))

Found 1832 images belonging to 6 classes.

Xception Model
Found 1832 images belonging to 6 classes.
Epoch 1/10


2023-04-27 07:10:34.166692: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - ETA: 0s - loss: 0.7194 - accuracy: 0.7571

2023-04-27 07:11:49.358738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 145s 2s/step - loss: 0.7194 - accuracy: 0.7571 - val_loss: 0.1734 - val_accuracy: 0.9640
Epoch 2/10
58/58 [==============================] - 142s 2s/step - loss: 0.2065 - accuracy: 0.9394 - val_loss: 0.0990 - val_accuracy: 0.9700
Epoch 3/10
58/58 [==============================] - 138s 2s/step - loss: 0.1163 - accuracy: 0.9694 - val_loss: 0.0462 - val_accuracy: 0.9896
Epoch 4/10
58/58 [==============================] - 141s 2s/step - loss: 0.0748 - accuracy: 0.9831 - val_loss: 0.0401 - val_accuracy: 0.9896
Epoch 5/10
58/58 [==============================] - 142s 2s/step - loss: 0.0679 - accuracy: 0.9809 - val_loss: 0.0240 - val_accuracy: 0.9956
Epoch 6/10
58/58 [==============================] - 142s 2s/step - loss: 0.0460 - accuracy: 0.9896 - val_loss: 0.0214 - val_accuracy: 0.9956
Epoch 7/10
58/58 [==============================] - 141s 2s/step - loss: 0.0375 - accuracy: 0.9885 - val_loss: 0.0136 - val_accuracy: 0.9989
Epoch 8/10
58/58 [======

2023-04-27 07:34:11.233816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 66s 1s/step - loss: 0.0322 - accuracy: 0.9885
Test accuracy: 0.9885371327400208
Test loss: 0.03216802328824997




2023-04-27 07:35:17.392984: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 68s 1s/step
Confusion Matrix:
 [[66 66 65 44 64 41]
 [64 71 72 41 59 40]
 [58 57 59 50 58 44]
 [41 42 47 36 42 31]
 [53 65 56 41 60 51]
 [45 46 47 27 43 40]]

Class 0 (Foi_Thong):
TP: 66, FP: 261, TN: 1225, FN: 280
Loss: 2.631

Class 1 (Hang_Kra_Rog_Phu_Phan_ST1):
TP: 71, FP: 276, TN: 1209, FN: 276
Loss: 3.527

Class 2 (Hang_Suea_Sakonnakhon_TT1):
TP: 59, FP: 287, TN: 1219, FN: 267
Loss: 3.318

Class 3 (Kroeng_Krawia):
TP: 36, FP: 203, TN: 1390, FN: 203
Loss: 2.578

Class 4 (Tanao_Si_Kan_Khaw_WA1):
TP: 60, FP: 266, TN: 1240, FN: 266
Loss: 3.612

Class 5 (Tanao_Si_Kan_Dang_RD1):
TP: 40, FP: 207, TN: 1377, FN: 208
Loss: 2.508

Overall Performance:
Accuracy: 0.727
Precision: 0.181
Recall: 0.181
F1-score: 0.181
Loss: 10.511

Class-specific Performance:

Class 0 (Foi_Thong) Performance:
Accuracy: 0.705
Precision: 0.202
Recall: 0.191
F1-score: 0.196
Loss: 2.631



Class 1 (Hang_Kra_Rog_Phu_Phan_ST1) Performance:
Accuracy: 0.699
Precision: 0.205
Recall

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (238,) + inhomogeneous part.

In [3]:
# Use the model to make predictions on new images
import matplotlib.pyplot as plt
import tensorflow.keras as keras

#figure 1
new_img = tf.keras.preprocessing.image.load_img('/data/test/test1.jpg', target_size=img_size)

plt.imshow(new_img)
plt.show()

new_img = tf.keras.preprocessing.image.img_to_array(new_img)
#print(new_img)
new_img = new_img / 255.0
new_img = tf.expand_dims(new_img, 0)

prediction = model.predict(new_img)
class_index = tf.argmax(prediction, axis=1)

print('\n')
predicted_label = labels[class_index[0]]
print('Predicted label:', predicted_label)

prediction_values = {}
for i, label in enumerate(labels):
    prediction_values[label] = round(prediction[0][i], 2)

print('Prediction values:', prediction_values)


1/1 [==============================] - 0s 61ms/step


Predicted label: Hang_Kra_Rog_Phu_Phan_ST1
Prediction values: {'Foi_Thong': 0.0, 'Hang_Kra_Rog_Phu_Phan_ST1': 1.0, 'Hang_Suea_Sakonnakhon_TT1': 0.0, 'Kroeng_Krawia': 0.0, 'Tanao_Si_Kan_Khaw_WA1': 0.0, 'Tanao_Si_Kan_Dang_RD1': 0.0}


In [5]:
print("Model Structure : \n")
model.summary()

Model Structure : 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                         32)                                       

                                                                                                  
 block4_sepconv1 (SeparableConv  (None, None, None,   188672     ['block4_sepconv1_act[0][0]']    
 2D)                            728)                                                              
                                                                                                  
 block4_sepconv1_bn (BatchNorma  (None, None, None,   2912       ['block4_sepconv1[0][0]']        
 lization)                      728)                                                              
                                                                                                  
 block4_sepconv2_act (Activatio  (None, None, None,   0          ['block4_sepconv1_bn[0][0]']     
 n)                             728)                                                              
                                                                                                  
 block4_se

 lization)                      728)                                                              
                                                                                                  
 add_4 (Add)                    (None, None, None,   0           ['block6_sepconv3_bn[0][0]',     
                                728)                              'add_3[0][0]']                  
                                                                                                  
 block7_sepconv1_act (Activatio  (None, None, None,   0          ['add_4[0][0]']                  
 n)                             728)                                                              
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, None, None,   536536     ['block7_sepconv1_act[0][0]']    
 2D)                            728)                                                              
          

 block9_sepconv3_act (Activatio  (None, None, None,   0          ['block9_sepconv2_bn[0][0]']     
 n)                             728)                                                              
                                                                                                  
 block9_sepconv3 (SeparableConv  (None, None, None,   536536     ['block9_sepconv3_act[0][0]']    
 2D)                            728)                                                              
                                                                                                  
 block9_sepconv3_bn (BatchNorma  (None, None, None,   2912       ['block9_sepconv3[0][0]']        
 lization)                      728)                                                              
                                                                                                  
 add_7 (Add)                    (None, None, None,   0           ['block9_sepconv3_bn[0][0]',     
          

                                                                                                  
 block12_sepconv2 (SeparableCon  (None, None, None,   536536     ['block12_sepconv2_act[0][0]']   
 v2D)                           728)                                                              
                                                                                                  
 block12_sepconv2_bn (BatchNorm  (None, None, None,   2912       ['block12_sepconv2[0][0]']       
 alization)                     728)                                                              
                                                                                                  
 block12_sepconv3_act (Activati  (None, None, None,   0          ['block12_sepconv2_bn[0][0]']    
 on)                            728)                                                              
                                                                                                  
 block12_s